# Unsloth 的微调样例代码

## 环境配置

系统为`Ubuntu 20.04`
CUDA为12.4，驱动是550.54.15

### 安装Conda
```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh
source ~/.bashrc
```

### 创建虚拟环境、添加一些镜像列表
```bash
# conda环境的镜像
conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/
conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main/
conda config --add channels https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/conda-forge/
conda config --add channels https://mirrors.bfsu.edu.cn/anaconda/pkgs/free/
conda config --add channels https://mirrors.bfsu.edu.cn/anaconda/pkgs/main/
conda config --add channels https://mirrors.bfsu.edu.cn/anaconda/cloud/conda-forge/
# pip镜像
pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
pip config set global.index-url https://mirrors.aliyun.com/pypi/simple/
pip config set global.index-url http://mirrors.cloud.tencent.com/pypi/simple
pip config set global.index-url http://pypi.douban.com/simple/
pip config set global.index-url https://mirrors.163.com/pypi/simple/
# 创建虚拟环境
conda create -n <环境名称> python=3.12
# 激活你创建的环境
conda activate <环境名称>
```

### 安装运行库
```bash
# pytorch安装
pip install --upgrade torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu124 --trusted-host download.pytorch.org
# 其他的依赖安装
pip install --upgrade torch==2.6.0 transformers BitsandBytes accelerate nltk numpy pandas tensorboardX evaluate scikit-learn sentence-transformers tiktoken deepspeed SentencePiece unsloth qwen-vl-utils[decord] nvitop trl -i https://mirrors.aliyun.com/pypi/simple/
# 如果用jupyter notebook 需要安装以下两个
conda install -n testllm ipykernel --update-deps --force-reinstall
conda install -n testllm IProgress
```

## 配置GPU

In [ ]:
import os, re, argparse
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # 这里替换成你想要的GPU序号

## 引入`unsloth`包，需要在`transformers`、`peft`和`trl`之前引入以方便`unsloth`进行加速

In [ ]:
import torch, unsloth
from unsloth import FastLanguageModel
from tqdm.auto import tqdm
import numpy as np
from trl import SFTConfig, SFTTrainer
from transformers import (
    TrainingArguments,
    LogitsProcessorList,
    InfNanRemoveLogitsProcessor,
    PreTrainedTokenizer,
    PreTrainedTokenizerBase,
    PreTrainedTokenizerFast,
    AutoTokenizer,
    AutoConfig,
    AutoModelForCausalLM,
    PreTrainedModel,
    PretrainedConfig,
    BitsAndBytesConfig,
    GenerationConfig,
    Seq2SeqTrainingArguments,
    Trainer,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)
from transformers.trainer import TRAINER_STATE_NAME
from transformers.utils.versions import require_version
from transformers.trainer_utils import SaveStrategy
from transformers.tokenization_utils import PaddingStrategy
from peft import PeftModel, LoraConfig, get_peft_model, TaskType
from peft.utils import (
    CONFIG_NAME,
    SAFETENSORS_WEIGHTS_NAME,
    TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING,
)
IGNORE_INDEX = -100 # 引入这个常数，有时候在梯度下降时用于屏蔽无需计算梯度的token

## 引入数据集和数据处理函数还有自定义的模板

In [ ]:
from datasets import Dataset, IterableDataset
from dataclasses import dataclass
# from 模板 import get_template_and_model_path
# from public_apis.file_rw import *
from typing import Union, Optional, List, Dict, Any, Literal, Sequence, Tuple

RANDOM_SEED = 42
IGNORE_INDEX=-100

TEMPLATE_DICT={
    'llama2':"A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. \n{user_query}\nAssistant:  ",
    'llama3':"<|start_header_id|>system<|end_header_id|>\nYou are a helpful assistant. <|eot_id|>\n\n<|start_header_id|>user<|end_header_id|>\n{user_query}<|eot_id|>\n\n<|start_header_id|>assistant<|end_header_id|>\n",
    'qwen2.5':"<|im_start|>system\nYou are a helpful assistant. <|im_end|>\n<|im_start|>user\n{user_query}<|im_end|>\n<|im_start|>assistant\n",
    'gpt2':"{user_query}\n"
}

MODEL_BASE_PATH='/data'

# MODEL_BASE_PATH='/data/lzy/models/LLM'

MODEL_NAME_DICT={
    'llama2-7b':os.path.join(MODEL_BASE_PATH,'llama','llama-2-7b-hf'),
    'llama2-13b':os.path.join(MODEL_BASE_PATH,'llama','llama-2-13b-hf'),
    'llama3.1-8b-i':os.path.join(MODEL_BASE_PATH,'llama','llama3.1-8b-instruct'),
    'llama3.1-8b':os.path.join(MODEL_BASE_PATH,'llama','llama3.1-8b'),
    'llama3.2-1b-i':os.path.join(MODEL_BASE_PATH,'llama','llama3.2-1B-Instruct'),
    'llama3.2-3b-i':os.path.join(MODEL_BASE_PATH,'llama','llama3.2-3B-Instruct'),
    'qwen2.5-0.5b-i':os.path.join(MODEL_BASE_PATH,'Qwen','Qwen2.5-0.5B-Instruct'),
    'qwen2.5-1.5b-i':os.path.join(MODEL_BASE_PATH,'Qwen','Qwen2.5-1.5B-Instruct'),
    'qwen2.5-3b-i':os.path.join(MODEL_BASE_PATH,'Qwen','Qwen2.5-3B-Instruct'),
    'qwen2.5-7b-i':os.path.join(MODEL_BASE_PATH,'Qwen','Qwen2.5-7B-Instruct'),
    'qwen2.5-14b-i':os.path.join(MODEL_BASE_PATH,'Qwen','Qwen2.5-14B-Instruct'),
    'qwen2.5-coder-0.5b-i':os.path.join(MODEL_BASE_PATH,'Qwen','Qwen2.5-Coder-0.5B-Instruct'),
    'qwen2.5-coder-1.5b-i':os.path.join(MODEL_BASE_PATH,'Qwen','Qwen2.5-Coder-1.5B-Instruct'),
    'qwen2.5-coder-3b-i':os.path.join(MODEL_BASE_PATH,'Qwen','Qwen2.5-Coder-3B-Instruct'),
    'qwen2.5-coder-7b-i':os.path.join(MODEL_BASE_PATH,'Qwen','Qwen2.5-Coder-7B-Instruct'),
    'qwen2.5-coder-14b-i':os.path.join(MODEL_BASE_PATH,'Qwen','Qwen2.5-Coder-14B-Instruct'),
    # 'gpt2l':os.path.join(MODEL_BASE_PATH,'openai-community','gpt2-large'),
    # 'gpt2':os.path.join(MODEL_BASE_PATH,'openai-community','gpt2')
}

def get_template_and_model_path(**arguments):
    model_name=str(arguments.get('model_name','llama3.1-8b-i'))
    model_name_or_path=MODEL_NAME_DICT[model_name]
    if model_name.startswith('qwen2.5'):
        template=TEMPLATE_DICT['qwen2.5']
    elif model_name.startswith('llama2'):
        template=TEMPLATE_DICT['llama2']
    elif model_name.startswith('gpt2'):
        template=TEMPLATE_DICT['gpt2']
    else:
        template=TEMPLATE_DICT['llama3']
    return model_name,model_name_or_path,template


# 自定义一种错误形式，主要用于防呆以及简化操作
class CustomDatasetTokenizerError(Exception):
    """Base class for custom exceptions in this module."""
    def __init__(self):
        # self.expression = expression
        self.message = "数据集元素的输入键值必须是以下组合：\n\t\"prompt\",\"query\",\"response\"\n\t\"instruction\",\"input\",\"output\",\n\t\"prompt\",\"completion\""
    def __str__(self):
        return self.message
    pass
# prompt_template应该是中间有个关键词叫做 `user_query` 
# 这个函数用于常规的微调（手搓，然后用`transformers`中的`trainer`或者`seq2seqtrainer`直接微调）
# 从载入的JSON文件一步变为令牌化的数据集样式
def process_tokens_tokenizer_functions(example: dict[str, str],tokenizer: Union[PreTrainedTokenizer,PreTrainedTokenizerBase,PreTrainedTokenizerFast,AutoTokenizer],prompt_template: str,train_mode: Literal["sft", "pt"] = "sft"):
    prompt=""
    completion=""
    if "prompt" in example.keys() and "query" in example.keys() and "response" in example.keys():
        prompt=prompt_template.format(user_query="{}\n{}\n".format(example["prompt"], example["query"]))
        completion=example["response"]
    elif "instruction" in example.keys() and "input" in example.keys() and "output" in example.keys():
        prompt=prompt_template.format(user_query="{}\n{}\n".format(example["instruction"], example["input"]))
        completion=example["output"]
    elif "prompt" in example.keys() and "completion" in example.keys():
        prompt=prompt_template.format(user_query="{}\n".format(example["prompt"]))
        completion=example["completion"]
    else:
        raise CustomDatasetTokenizerError()
    # SFT样本构建
    model_inputs = {"input_ids": [], "attention_mask": [], "labels": []}
    # 找出非标签部分
    example_str = "{}{}".format(prompt, completion)
    input_id = tokenizer.encode(example_str)
    # 训练数据集末尾带上截止符号
    if input_id[-1] != tokenizer.eos_token_id:
        input_id.append(tokenizer.eos_token_id)
    input_id_before_label = tokenizer.encode(prompt)
    label = input_id.copy()
    # SFT只需要label部分计算梯度，pt需要所有部分计算梯度
    if train_mode == "sft":
        label[: len(input_id_before_label)] = [IGNORE_INDEX] * len(
            input_id_before_label
        )
    attention_mask = [1] * len(input_id)
    # 放入输入字典中
    if "qid" in example.keys():
        model_inputs["qid"] = example["qid"]
    elif "ID" in example.keys():
        model_inputs["qid"] = example["ID"]
    model_inputs["input_ids"] = input_id
    model_inputs["attention_mask"] = attention_mask
    model_inputs["labels"] = label
    return model_inputs
# 直接处理成`trl`可接受的格式（直接文本就行，剩下的交给剩下）
def process_dataset_functions(example: dict[str, str]):
    prompt=""
    completion=""
    if "prompt" in example.keys() and "query" in example.keys() and "response" in example.keys():
        prompt="{}\n{}\n".format(example["prompt"], example["query"])
        completion=example["response"]
    elif "instruction" in example.keys() and "input" in example.keys() and "output" in example.keys():
        prompt="{}\n{}\n".format(example["instruction"], example["input"])
        completion=example["output"]
    elif "prompt" in example.keys() and "completion" in example.keys():
        prompt="{}\n".format(example["prompt"])
        completion=example["completion"]
    else:
        raise CustomDatasetTokenizerError()
    return {"prompt":prompt,"completion":completion,"text":"### Instruction:\n {}\n ### Response:\n {}\n".format(prompt,completion),"ground_truth":completion}

## 加载模型模板

In [ ]:
model_name, model_name_or_path, template = get_template_and_model_path(
    model_name="llama3.1-8b-i"
)

# Load model
max_length=2048
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name_or_path,
    max_seq_length=max_length,
    dtype=None,  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
    # load_in_4bit=True,  # Use 4bit quantization to reduce memory usage. Can be False
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

# Do model patching and add fast LoRA weights
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj"
    ],
    lora_alpha=32,
    lora_dropout=0.1,  # Dropout = 0 is currently optimized
    bias="none",  # Bias = "none" is currently optimized
    use_gradient_checkpointing=True,
    random_state=3407,
)




## 训练

输入的数据集得的结构是以下三种类型的：

1. 键值同时包含（第一优先级）： `prompt`,`query`,`response`
2. 键值同时包含（第二优先级）： `instruction`,`input`,`output`
3. 键值同时包含（第三优先级）： `prompt`,`completion`

In [ ]:
from datasets import load_dataset
dataset = load_dataset("<你的数据集位置>", split="train")
print(dataset[0])
# dataset_processed=dataset.map(lambda example: {"prompt": "{}\n{}\n".format(example['instruction'],example['input']), "completion":example['output'] ,"text":"### Prompt:\n {}\n{}\n ### Completion:\n {}\n".format(example['instruction'],example['input'],example['output'])})
dataset_processed=dataset.map(lambda example: process_dataset_functions(example))
dataset_processed=dataset_processed.remove_columns(dataset.column_names)
print(dataset_processed[0])

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_processed,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 20, # Set this for 1 full training run.
        save_steps=1000,
        save_strategy=SaveStrategy.STEPS,
        # max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
        completion_only_loss=True,
    ),
    # formatting_func= lambda example: "### Prompt:\n {}\n ### Completion:\n {}\n".format(example['prompt'],example['completion'])
)
trainer.train()
trainer.save_model()